In [38]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
import cv2
import math
import utils
import imutils


def disp_image(image):
    plt.figure(figsize=(25, 25))
    plt.imshow(image)
    plt.show()

In [206]:
orig = cv2.imread("/home/dan/Downloads/poulespng/poule-7.pdf.png", cv2.IMREAD_UNCHANGED)
tmpl = cv2.imread("./data/pngs/poule-front-content-tmpl.png", cv2.IMREAD_UNCHANGED)

blur_amount = (13, 13)
crop_margin_px = 80

debug = True

# Crop inward, there won't be anything at the edges except for scanning artifacts
orig = orig[crop_margin_px:-crop_margin_px, crop_margin_px:-crop_margin_px]

img_gray = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
thresh = cv2.GaussianBlur(thresh, blur_amount, 0)
th, thresh = cv2.threshold(img_gray, 254, 255, cv2.THRESH_BINARY)
thresh = cv2.erode(thresh, None, iterations=3)

corner_crop_px = 300

thresh = utils.crop_to_content(255 - thresh)

tl = thresh[
    0:corner_crop_px, 
    0:corner_crop_px
]
bl = thresh[
    thresh.shape[0] - corner_crop_px:thresh.shape[0],
    0:corner_crop_px
]
tr = thresh[
    0:corner_crop_px, 
    thresh.shape[1] - corner_crop_px:thresh.shape[1]
]
br = thresh[
    thresh.shape[0] - corner_crop_px:thresh.shape[0],
    thresh.shape[1] - corner_crop_px:thresh.shape[1]
]

#     copy = image.copy()
#     # these are coords yay!
#     cnts = cv2.findContours(copy, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     cnts = imutils.grab_contours(cnts)
#     copy = cv2.cvtColor(copy, cv2.COLOR_BGR2RGB)
# #     cv2.drawContours(copy, cnts, -1, (0, 255, 0), 1)
#     areas = [cv2.contourArea(c) for c in cnts]
# #     areas = [a for a in areas if a > 200.0 and a < 2000.0]

def findHoriLines(image, max_height=20, min_width=100, debug=False):
    # these are coords yay!
    cnts = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    areas = [cv2.contourArea(c) for c in cnts]
    polys = [cv2.approxPolyDP(c, 3, True) for c in cnts]
    rects = [cv2.boundingRect(p) for p in polys]
    
    result = []
    for r in rects:
        x, y, w, h = r

        if abs(h) > max_height: continue
        if abs(w) < min_width: continue

        result.append(r)
    
    # Sort top to bottom
    result.sort(key=lambda r: r[1])
    
    if (debug is True):
        copy = image.copy()
        copy = cv2.cvtColor(copy, cv2.COLOR_BGR2RGB)
        cv2.drawContours(copy, cnts, -1, (0, 255, 255), 2)
        
        for x, y, w, h in result:
            cv2.rectangle(copy, (x, y), (x + w, y + h), (255, 0, 0), 5)
        disp_image(copy)

    return result

[ first, last ] = findHoriLines(thresh, debug=False)

print(first, len(thresh[first[0]]))
print(last, len(thresh[first[1]]))

# How to find corners
# for idx in range(first[1], first[1] + first[3]):
#     y_px = thresh[idx][22:]
#     print(y_px)


# NOTE x: 0, y: 1, w: 2, h: 3
# So we go over each row of our first line



print("tl", find_corner_of_line(thresh, first[0]               , first[1], first[3]))
print("tr", find_corner_of_line(thresh, first[0] + first[2] - 1, first[1], first[3]))
print("bl", find_corner_of_line(thresh, last[0]                , last[1],  last[3]))
print("br", find_corner_of_line(thresh, last[0]  + last[2]  - 1, last[1],  last[3]))

# TODO now find most extreme pixels
# The location that we return should remap onto the original image
# So tl = found extreme corner = tl_xc.y + any_offset coming from other cropping
# Or we just don't care:
# - Crop naive to strip artifacts  ✓
# - Crom to content                ✓
# - Find corners                   ✓
# - hindHomography()
# - persectiveWarp()
# - Resize
# - ...
# - Profit

image_arr [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
tl [73 13]
br [3318 2293]
height 3245
width 2280
(24, 0, 2218, 18) 2280
(5, 3104, 2221, 17) 2280
tl (24, 0)
tr (2241, 10)
bl (5, 3104)
br (2225, 3113)
